In [6]:
from transformers import AutoTokenizer ,GPT2LMHeadModel, AdamW,GPT2TokenizerFast
from torch.utils.data import Dataset, DataLoader
import torch
import json
import numpy as np
from tqdm import tqdm
import pandas as pd
import re
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import json
import re

In [2]:
C_TKN = "<unused3>"#문맥토큰
Q_TKN = "<unused4>" #질문토큰
A_TKN = "<unused5>" #답변 토큰
BOS = '<s>'#begin of sequence
EOS = '</s>'#end of sequence
MASK = '<unused0>'# mask token
SENT = '<unused1>'#문장토큰
PAD = '<pad>'

tokenizer = AutoTokenizer.from_pretrained("skt/kogpt2-base-v2", bos_token=BOS, eos_token=EOS, unk_token="<unk>", pad_token=PAD, mask_token=MASK,)


In [3]:
with open("./Similar_data_klue_large_KoELECTRA.json","r",encoding="utf-8") as f:
    data = json.load(f)

In [4]:
Similar_raw = []

for item in data["data"]:
    for para in item["para"]:
        context = para["context"]
        for labels in para["label"]:
            label = labels["sequence"]
            token_str = labels["token_str"]
            Similar_raw.append((context,label,token_str))

print(len(Similar_raw))
print(Similar_raw[0])
print(Similar_raw[1])


19709
('쓸쓸스러운 붉은 감 닙이 죽어가는 생물처럼 여기저기 휘둘러서 헛날닐 제 말 없이 오는 가을바람이 따뜻한 나의 가슴을 근질이고 지내감에, 나도 모르는 쓸쓸한 비애가 나의 두 눈을 공연히 울고 싶게 하였다.', '쓸쓸스러운 붉은 감 닙이 죽어가는 것 처럼 여기저기 휘둘러서 헛날닐 제 말 없이 오는 가을바람이 따뜻한 나의 가슴을 근질이고 지내감에, 나도 모르는 쓸쓸한 비애가 나의 두 눈을 공연히 울고 싶게 하였다.', '것')
('쓸쓸스러운 붉은 감 닙이 죽어가는 생물처럼 여기저기 휘둘러서 헛날닐 제 말 없이 오는 가을바람이 따뜻한 나의 가슴을 근질이고 지내감에, 나도 모르는 쓸쓸한 비애가 나의 두 눈을 공연히 울고 싶게 하였다.', '쓸쓸스러운 붉은 감 닙이 죽어가는 나무 처럼 여기저기 휘둘러서 헛날닐 제 말 없이 오는 가을바람이 따뜻한 나의 가슴을 근질이고 지내감에, 나도 모르는 쓸쓸한 비애가 나의 두 눈을 공연히 울고 싶게 하였다.', '나무')


In [7]:
train_data_raw , test_data_raw = train_test_split(Similar_raw,test_size = 0.2)
print(len(train_data_raw),len(test_data_raw))

15767 3942


In [8]:
train_data = pd.DataFrame(train_data_raw)
test_data = pd.DataFrame(test_data_raw)
print(len(train_data))
train_data.rename(columns={0: 'context',1:'label',2:'token'}, inplace=True)
test_data.rename(columns={0: 'context',1:'label',2:'token'}, inplace=True)


15767


In [9]:
#데이터 처리 클래스
class Similar_Dataset(Dataset):
    def __init__(self, chats, max_len=512):  # 데이터셋의 전처리를 해주는 부분
        self._data = chats
        self.max_len = max_len
        self.bos= BOS
        self.eos = EOS
        self.pad = PAD
        self.tokenizer = tokenizer
    def __len__(self):  
        return len(self._data)

    def __getitem__(self, idx):  # 로드한데이터를 차례차례 DataLoader로 넘김
        turn = self._data.iloc[idx]
        l = turn["label"]  # label을 가져온다.
        c = turn["context"]  # 문맥을 가져온다.
        
        c_toked = self.tokenizer.tokenize(self.bos + c + self.eos)
        c_len = len(c_toked)
        
        l_toked = self.tokenizer.tokenize(self.bos + l + self.eos)
        l_len = len(l_toked)

        #최대길이 초과시 처리
        if c_len> self.max_len:
            c_len = self.max_len
            c_toked = c_toked[:c_len]
       
        if l_len> self.max_len:
            l_len = self.max_len
            l_toked = l_toked[:l_len]

        labels_ids = self.tokenizer.convert_tokens_to_ids(l_toked)        
       
        while len(labels_ids) < self.max_len: # 최대길이만큼 PADDING
            labels_ids += [self.tokenizer.pad_token_id]

        token_ids = self.tokenizer.convert_tokens_to_ids(c_toked)
      
        while len(token_ids) < self.max_len: # 최대길이만큼 PADDING
            token_ids += [self.tokenizer.pad_token_id]

        #질문+답변, 마스크, 답변
        return (token_ids, labels_ids)

In [10]:
def collate_batch(batch):
    data = [item[0] for item in batch]
    label = [item[1] for item in batch]
    return torch.LongTensor(data), torch.LongTensor(label)

In [11]:
train_set = Similar_Dataset(train_data, max_len=512)
loader = DataLoader(train_set, batch_size=4,  num_workers=0, shuffle=True, collate_fn=collate_batch,)
len(loader)

3942

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [13]:
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
model.to(device)
model.train()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [16]:
from transformers import get_linear_schedule_with_warmup
epochs = 10
optimizer = AdamW(model.parameters(), lr=5e-5)
total_steps = len(loader) * epochs

# 스케줄러 설정
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=total_steps)

# 학습 과정
for epoch in range(epochs):
    model.train()
    total_loss = 0
    progress_bar = tqdm(loader, desc="Epoch {:1d}".format(epoch+1), leave=False, disable=False)
    for batch in progress_bar:
        optimizer.zero_grad()
        inputs, labels = tuple(t.to(torch.device("cuda" if torch.cuda.is_available() else "cpu")) for t in batch)
        
        outputs = model(input_ids = inputs, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item() / len(batch))})
        
    print("Epoch {}/{} - Loss: {:.4f}".format(epoch+1, epochs, total_loss / len(loader)))

Epoch 1:   0%|          | 0/3942 [00:00<?, ?it/s]

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Epoch 1/10 - Loss: 0.3947


Epoch 2/10 - Loss: 0.2298


Epoch 3/10 - Loss: 0.1404


...

Epoch 6/10 - Loss: 0.0769


Epoch 7/10 - Loss: 0.0693


Epoch 8/10 - Loss: 0.0631


Epoch 9/10 - Loss: 0.0574


Epoch 10/10 - Loss: 0.0523


In [17]:
model.save_pretrained("./Similar_gpt2_model1")
tokenizer.save_pretrained("./Similar_gpt2_model1")

('./Similar_gpt2_model1/tokenizer_config.json',
 './Similar_gpt2_model1/special_tokens_map.json',
 './Similar_gpt2_model1/vocab.json',
 './Similar_gpt2_model1/merges.txt',
 './Similar_gpt2_model1/added_tokens.json',
 './Similar_gpt2_model1/tokenizer.json')

In [25]:
from transformers import BartForConditionalGeneration
def generate_sequence(context, model, tokenizer, device, num_sequences=2):
    model.eval()  # 모델을 평가 모드로 설정

    # 입력 문맥 토크나이징
    input_ids = tokenizer.encode(BOS + context + EOS, return_tensors="pt").to(device)

    # 모델을 사용하여 출력 생성
    output_sequences = model.generate(
        input_ids=input_ids,
        max_length=512,
        temperature=0.7,
        top_k=50,
        top_p=0.9,
        repetition_penalty=2.0,
        do_sample=True,
        num_return_sequences=num_sequences
    )

    generated_sequences = []
    for generated_sequence in output_sequences:
        # 생성된 시퀀스를 디코딩하여 텍스트로 변환
        decoded_text = tokenizer.decode(generated_sequence, skip_special_tokens=True)
        generated_sequences.append(decoded_text)
    
    return generated_sequences

model_path = "Similar_bart_model_05_29"


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def multiple_replace(text, replacements):
    for old, new in replacements.items():
        text = text.replace(old, new)
    return text

replacements = { BOS: "", EOS: "", PAD: ""}

tokenizer = AutoTokenizer.from_pretrained(model_path)
model =BartForConditionalGeneration.from_pretrained(model_path).to(device)
context = test_data["context"][4]
context = '저 거울은 은화처럼 빛났다'
print("원본:", context)

generated_sentences = generate_sequence(context, model, tokenizer, device, num_sequences=2)

for idx, sentence in enumerate(generated_sentences):
    sentence = multiple_replace(sentence, replacements)
    print(f"생성 {idx+1}:", sentence)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


원본: 저 거울은 은화처럼 빛났다
생성 1: 
생성 2: 
